Import packages

In [ ]:
import pandas as pd
import string
import spacy
from tqdm import tqdm

tqdm.pandas()
print("Imports & tqdm ready")


Imports & tqdm ready


Load/Prepare data

In [ ]:
df = pd.read_csv("Dutch_Migration_News.csv")
print(df.shape)
print(df.columns)
df = df.drop(columns=['Unnamed: 0'])

TITLE_COL = "title"
BODY_COL  = "body"

df[TITLE_COL] = df[TITLE_COL].astype(str)
df[BODY_COL]  = df[BODY_COL].astype(str)

df["text"] = df[TITLE_COL] + " " + df[BODY_COL]
print("Data loaded & text combined")
df.head(10)


(12853, 7)
Index(['Unnamed: 0', 'outlet', 'date', 'year', 'authors', 'title', 'body'], dtype='object')
Data loaded & text combined


,outlet,date,year,authors,title,body,text
0,NRC Online,NaN,2000,Merijn de Waal,‘El Chino’ bracht Peru met harde hand en veel ...,Als kind op het schoolplein wordt de Peruaanse...,‘El Chino’ bracht Peru met harde hand en veel ...
1,NRC Online,NaN,2000,Juurd Eijsvoogel,"‘Wij vinden onszelf niet rechts’, zegt vertrek...",Wie verwachtte dat het opinieweekblad EW (voor...,"‘Wij vinden onszelf niet rechts’, zegt vertrek..."
2,NRC Online,NaN,2001,Casper van der Veen,Israël staat toe dat humanitaire hulp wordt ge...,Israël zal humanitaire hulp die vanuit Egypte ...,Israël staat toe dat humanitaire hulp wordt ge...
3,NRC Online,NaN,2001,Bas Blokker,‘Als Amerikanen zijn we verre van volmaakt’ ‘A...,"‘Mag ik wat Nederlandse exemplaren meenemen”, ...",‘Als Amerikanen zijn we verre van volmaakt’ ‘A...
4,AD Print,NaN,2001,ROBIN ROTMAN,INTERVIEW Eric Fuanani moet zijn voetbaldroom ...,ROBIN ROTMAN\nKLUNDERT\nEric Fuanani (17) uit ...,INTERVIEW Eric Fuanani moet zijn voetbaldroom ...
5,NRC Online,NaN,2001,Pim van den Dool,Haagse consensus over migratiebeperking vervli...,De politiek is verdeeld over de vraag of Neder...,Haagse consensus over migratiebeperking vervli...
6,Trouw Print,NaN,2001,jacqueline wouda,Meppel doet niet moeilijk over komst asielzoek...,"Solidariteit, gastvrijheid, het waren sleutelw...",Meppel doet niet moeilijk over komst asielzoek...
7,Trouw Print,NaN,2001,Ina Reitzema,Armeens gezin dreigt uit azc Ten Boer te worde...,Ten Boer - Een Armeens gezin dreigt aanstaande...,Armeens gezin dreigt uit azc Ten Boer te worde...
8,Trouw Print,NaN,2002,Unknown Authors,Integratie is een kwestie van geduld Integrati...,Door Thea van Beek\n Amsterdam\n Immigratie en...,Integratie is een kwestie van geduld Integrati...
9,Trouw Print,NaN,2002,Unknown Authors,'Bij asielbeleid helpen emoties je niet verder...,Door Paul Koopman en\n Lianne Sleutjes\n Den H...,'Bij asielbeleid helpen emoties je niet verder...


Load Spacy model (Dutch)

In [ ]:
# Run once if needed:
#!python -m spacy download nl_core_news_sm
nlp = spacy.load("nl_core_news_sm")
print("SpaCy model loaded")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 61.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
SpaCy model loaded


Preprocessing function

In [13]:
def clean_text_spacy(doc):
    include_pos = ["NOUN", "PROPN", "ADJ", "VERB"]

    text = " ".join(
        t.lemma_.strip()
        for t in doc
        if t.pos_ in include_pos and not t.is_stop
    )

    text = text.translate(str.maketrans(" ", " ", string.punctuation))
    text = text.translate(str.maketrans(" ", " ", string.digits))
    text = text.lower()

    return text

print("Preprocessing function defined")


Preprocessing function defined


NLP + Progress (slowest step)

In [ ]:
df["text_nlp"] = df["text"].progress_apply(lambda x: nlp(x))
print("SpaCy NLP completed")


 14%|█▍        | 1785/12853 [02:50<12:39, 14.57it/s]

Clean text & tokens

In [ ]:
df["clean_text"] = df["text_nlp"].apply(clean_text_spacy)
df["clean_tokens"] = df["clean_text"].apply(lambda x: x.split())
print("Clean text & tokens created")


Sanity check

In [ ]:
df.head()